In [138]:
path = "COVID-Dialogue-Dataset-English.txt"
f = open(path,encoding="UTF-8")
l1 = f.read()
l2 = l1.split("id=")
p2d = []
d2p = []

def clean_sentence(str):
    str = str.replace("\n", "").replace("/","").replace("...", "").replace("*","").replace(".", " . ")

    return str
for item in l2:
    if len(item) != 0:
        temp = item.split("Dialogue\nPatient:")
        if len(temp) > 1:
            item2 = temp[1]
            item3 = item2.split("Patient:")
            l = []
            if len(item3) != 0:
                for item4 in item3:
                    item5 = item4.split("Doctor:")
                    item5 = [clean_sentence(item6) for item6 in item5]
                    l += item5
            for i in range(1, len(l), 2):
                p2d.append((l[i - 1].strip("\n"), l[i].strip("\n")))
                if i + 1 < len(l):
                    d2p.append((l[i].strip("\n"), l[i + 1].strip("\n")))
data = p2d + d2p
data[0]

('Hello doctor, I get a cough for the last few days, which is heavy during night times .  No raise in temperature but feeling tired with no travel history .  No contact with any Covid-19 persons .  It has been four to five days and has drunk a lot of Benadryl and took Paracetamol too .  Doctors have shut the OP so do not know what to do? Please help . ',
 'Hello, I understand your concern .  I just have a few more questions . Does your cough has phlegm? Any other symptoms like difficulty breathing? Any other medical condition such as asthma, hypertension? Are you a smoker? Alcoholic beverage drinker?')

In [0]:
import json
json_str = json.dumps(data)
f = open("dia_data.txt", "w")
json.dump(json_str,f)

In [0]:
import json
f = open("dialog_data.txt", "w")
json_str = json.dumps(data)
json.dump(json_str,f)

In [161]:
dictionary = {}
MAX_LENGTH = 50
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from itertools import chain

sw = stopwords.words('english')
nltk.download('stopwords')
punct = punctuation
reverse_flag = False
stopword_flag = False
shorten_sentences_flag = True
words_min_times = 0

def shorten_sentences(str):
    strs = str.split(".")
    result = ""
    length = 0
    for i in range(len(strs)):
        item = strs[i]
        if len(item.split(" ")) + length < MAX_LENGTH:
            if i < len(strs) - 1:
                result += item + ". "
            else:
                result += item
                return result
            length = len(result.split(" "))
        else:
            break
    return result

def clean_data(data):
    for item in data:
        for item2 in item:
            words = word_tokenize(item2)
            for word in words:
                dictionary.setdefault(word,0)
                dictionary[word] += 1
    data2 = []
    for item in data:
        temp = []
        flag = 0
        if "www" in item[0] or "www" in item[1]:
            continue
        for item2 in item:
            item2 = " ".join([token for token in word_tokenize(item2) if dictionary[token] > words_min_times])
            
            if reverse_flag and flag == 1:
                item2 = " ".join(reversed(item2.split(" ")))
            if stopword_flag:
                item2 = " ".join([token for token in word_tokenize(item2) if token.lower() not in chain(punct, sw)])
            if shorten_sentences_flag:
                item2 = shorten_sentences(item2)
            temp.append(item2)
            flag += 1
        temp = tuple(temp)
        if temp[0] == "" or temp[1] == "":
            continue
        data2.append(temp)
    return data2
data2 = clean_data(data)
data2[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


('Hello doctor , I get a cough for the last few days , which is heavy during night times .  No raise in temperature but feeling tired with no travel history .  No contact with any Covid-19 persons . ',
 'Hello , I understand your concern .  I just have a few more questions . ')

In [162]:
shorten_sentences("sha sia ajss. sdasds ss. ds ssa aa a .")

'sha sia ajss.  sdasds ss.  ds ssa aa a . '

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"([.!?\'])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang):
    print("Reading lines...")
    '''
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    '''
    pairs = [(normalizeString(item[0]), normalizeString(item[1])) for item in data2]
    # Reverse pairs, make Lang instances


    lang = Lang(lang)

    return lang, pairs

In [0]:
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [168]:
def prepareData(lang):
    lang, pairs = readLangs(lang)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        #print(pair)
        lang.addSentence(pair[0])
        lang.addSentence(pair[1])
    print("Counted words:")
    print(lang.name, lang.n_words)
    return lang, pairs


lang, pairs = prepareData('lang')
print(random.choice(pairs))

Reading lines...
Read 556 sentence pairs
Trimmed to 556 sentence pairs
Counting words...
Counted words:
lang 3695
("hi .  this covid-19 outbreak is scary .  i got screened this week and it was negative .  but prior to screening i had a week of continuous dry coughs and also throat was sore .  i 've put myself in a quarantine .", 'screening test if .  if you develop symptoms of dry cough , fever andor shortness of breath , consider re testing since the test may convert to a positive test if infected .  other causes should be considered including an allergic reaction .')


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0])
    target_tensor = tensorFromSentence(lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print("the size of data is:", len(pairs))
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        p1 = pair[1].split(" ")
        if reverse_flag:
            p1 = reseverd(p1)
        print('=', " ".join(p1))
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        if reverse_flag:
            output_words = reseverd(output_words)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [179]:
hidden_size = 256
encoder1 = EncoderRNN(lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, lang.n_words, dropout_p=0.1).to(device)
pairs = pairs
trainIters(encoder1, attn_decoder1, 20000, print_every=200)

the size of data is: 556
0m 54s (- 89m 31s) (200 1%) 5.0048
1m 43s (- 84m 18s) (400 2%) 4.3669
2m 37s (- 84m 44s) (600 3%) 4.9065
3m 32s (- 84m 57s) (800 4%) 4.7602
4m 27s (- 84m 37s) (1000 5%) 4.7731
5m 21s (- 84m 3s) (1200 6%) 4.5935
6m 17s (- 83m 37s) (1400 7%) 4.6620
7m 13s (- 83m 3s) (1600 8%) 4.5714
8m 10s (- 82m 35s) (1800 9%) 4.6780
9m 7s (- 82m 4s) (2000 10%) 4.5296
10m 3s (- 81m 21s) (2200 11%) 4.3195
11m 0s (- 80m 42s) (2400 12%) 4.1823
11m 59s (- 80m 14s) (2600 13%) 4.2872
12m 55s (- 79m 26s) (2800 14%) 4.1557
13m 52s (- 78m 38s) (3000 15%) 4.2057
14m 48s (- 77m 45s) (3200 16%) 4.0600
15m 45s (- 76m 56s) (3400 17%) 4.0245
16m 40s (- 75m 59s) (3600 18%) 4.0060
17m 38s (- 75m 13s) (3800 19%) 3.9883
18m 35s (- 74m 22s) (4000 20%) 3.8575
19m 32s (- 73m 32s) (4200 21%) 3.8596
20m 31s (- 72m 46s) (4400 22%) 3.8330
21m 31s (- 72m 3s) (4600 23%) 3.6191
22m 34s (- 71m 29s) (4800 24%) 3.6600
23m 41s (- 71m 5s) (5000 25%) 3.7357
24m 46s (- 70m 32s) (5200 26%) 3.6613
25m 48s (- 69m 47s

In [180]:
evaluateRandomly(encoder1, attn_decoder1)

> hi , do not worry so much .  you will be absolutely alright .  give it some time .  if you are not getting better we can do laryngoscopy and check if there is anything wrong with the larynx .
= yeah if it does n't get better laryngoscopy would be perfect .  okay , i 'll give it some time thank you ! i try to eat vegetables and fruits .  i do n't eat much junk food that much .
< yeah if . do n't n't you n't be an chest n't do n't get  and then some sleep to be in testing .  try to do this platform .  and do this platform . <EOS>

> i have had constant chest inf was hospitalised for 4 days said they think phnewmonia out now still on lots of anti biotics and steroids however i 'm still sweating have right lower back pain and still really coughing and short of breath .
= helloyes according to the history it might be pneumonia .  continue the current treatment .  regardsdr .  jolanda
< helloyes according to the history it , the pneumonia , you should continue the it it  continue your it c

In [181]:
pairs

[('hello doctor , i get a cough for the last few days , which is heavy during night times .  no raise in temperature but feeling tired with no travel history .  no contact with any covid-19 persons .',
  'hello , i understand your concern .  i just have a few more questions .'),
 ('thank you doctor , i have phlegm but not a lot .  a tiny amount comes out most of the time .  i have no difficulty in breathing .  no medical conditions and not a smoker nor a drinker .',
  'hi , i would recommend you take n-acetylcysteine 200 mg powder dissolved in water three times a day .  you may also nebulize using pnss ( saline nebulizer ) three times a day .  this will help the phlegm to come out .'),
 ('hello doctor , i am suffering from coughing , throat infection from last week .  at that time fever did not persist and also did not felt any chest pain .  two days later , i consulted with a doctor .',
  'hello , i can understand your concern .  in my opinion , you should get done a chest x-ray and c

In [0]:
lang.index2word

In [0]:
lang.n_words

In [0]:
pairs